In [2]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

Finding the meetings in the page:

AgendaCenter: table_rows = driver.find_elements(By.CSS_SELECTOR, "tr[class*=catAgendaRow") (get the href in a next step)

Boarddocs: all_meetings = driver.find_elements(By.CSS_SELECTOR, "a[class*='icon prevnext meeting") (boarddocs is so specific, it may be best to leave it as standalone code instead of converting it to be modular)

CivicClerk: all_meetings = driver.find_elements(By.CSS_SELECTOR,"li[class*='cpp-MuiListItem-container'") 

CivicWeb: all_meetings = driver.find_elements(By.CSS_SELECTOR,"a[class*='list-link'")

Document Center: agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[class*=pdf") (get href in the next step)

eScribe: future_meetings = driver.find_elements(By.CSS_SELECTOR,"div[class*='upcoming-meeting-container'")

Granicus v2: table_rows = driver.find_elements(By.CSS_SELECTOR, 'div[class*=RowTop') (get the specific meeting links in the next step)

Granicus: table_rows=driver.find_elements(By.CSS_SELECTOR,"tr[class*=listingRow") (get the href in a following step)

* Laserfiche: meeting_folders = driver.find_elements(By.CSS_SELECTOR,"tr")

Legistar: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr[id*=ctl00_')

* MeetingsTable: all_rows = driver.find_elements(By.CSS_SELECTOR,"tr")[1:]

* NovusAgenda: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

* OnBase: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

* PHP Table: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr')

Prime Gov: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr[role*=row')

* Albemarle: table_rows = driver.find_elements(By.CSS_SELECTOR, "tr")

Alleghaney: all_agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[class*='attachment-link'") (finds the links right away)

Bath: pc_links = driver.find_elements(By.CSS_SELECTOR,"a[title*='Planning'") , bos_links = driver.find_elements(By.CSS_SELECTOR,"a[title*='Board of Supervisors'")

* Bland: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")



In [ ]:
"""Tazewell County"""
def tazewell_county(url, government_body):
    driver.get(url)
    time.sleep(10)
    messages = []
    all_rows = driver.find_elements(By.CSS_SELECTOR,"li")
    meeting_rows = []
    for item in all_rows:
        if "2024" in item.text:
            meeting_rows.append(item)
    for item in meeting_rows:
        future_meeting = check_meeting_date(item.text)
        if future_meeting == True:
            try:
                #agenda is listed first, so can just return the first search result
                agenda_link = item.find_element(By.CSS_SELECTOR,"a")
                agenda_url = agenda_link.get_attribute("href")
                driver.get(agenda_url)
                time.sleep(10)
                agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
                agenda_search = search_agenda_for_keywords(agenda_content)
                if agenda_search != []:
                    messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Tazewell County " + government_body + ". " + agenda_url)
            except:
                continue
            #if there's no agenda posted yet and the initial agenda link search fails, just move on to the next meeting
    return messages

In [2]:
driver.get("https://tazewellcountyva.org/government/boards-and-commissions/board-of-supervisors/")

In [35]:
all_rows = driver.find_elements(By.CSS_SELECTOR,"li")

In [37]:
meeting_rows = []
for item in all_rows:
    if "2024" in item.text:
        meeting_rows.append(item)

In [42]:
messages=[]
government_body='Board of Supervisors'
for item in meeting_rows:
    future_meeting = check_meeting_date(item.text)
    if future_meeting == True:
        try:
                #agenda is listed first, so can just return the first search result
            agenda_link = item.find_element(By.CSS_SELECTOR,"a")
            agenda_url = agenda_link.get_attribute("href")
            driver.get(agenda_url)
            time.sleep(10)
            agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
            agenda_search = search_agenda_for_keywords(agenda_content)
            if agenda_search != []:
                messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Tazewell County " + government_body + ". " + agenda_url)
        except:
            continue

StaleElementReferenceException: Message: The element with the reference 47b498f0-6501-4a5c-9102-5f3b4622bea9 is not known in the current browsing context
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:383:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:259:20
cloneObject@chrome://remote/content/marionette/json.sys.mjs:59:24
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:289:16
json.deserialize@chrome://remote/content/marionette/json.sys.mjs:293:10
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:73:30


In [ ]:
future_meeting = check_meeting_date(item.text)

In [33]:
#checking the 08/05 webscraping run
tazewell_county("https://tazewellcountyva.org/government/boards-and-commissions/board-of-supervisors/", "Board of Supervisors")

StaleElementReferenceException: Message: The element with the reference f3cd54ea-0cda-4ded-90f0-b0535e234697 is not known in the current browsing context
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:383:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:259:20
cloneObject@chrome://remote/content/marionette/json.sys.mjs:59:24
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:289:16
json.deserialize@chrome://remote/content/marionette/json.sys.mjs:293:10
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:73:30
